In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.utils as vutils
import matplotlib.pyplot as plt
import os
import imageio
import numpy as np

In [14]:
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.__version__)

True
12.4
2.4.1+cu124


In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [16]:
numberofEpochs = 100
batchSize = 32
learningRateGenerator = 0.0002
learningRateDiscriminator = 0.0002
imageSize = 28

In [17]:
transform = transforms.Compose([
    transforms.Resize(imageSize), 
    transforms.ToTensor(), 
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = dsets.MNIST(root='./data', download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size = batchSize, shuffle=True)

In [18]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(128, 1, 4, 2, 1, bias=False),
            nn.Tanh()
        )
    
    def forward(self, input):
        return self.main(input)

In [19]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(1, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(256, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input).view(-1)

In [20]:
discriminator = Discriminator().to(device)
generator = Generator().to(device)

In [21]:
criterion = nn.BCELoss()
optimizerDiscriminator = optim.Adam(Discriminator().parameters(), lr = learningRateDiscriminator, betas = (0.5, 0.999))
optimizerGenerator = optim.Adam(Generator().parameters(), lr = learningRateGenerator, betas = (0.5, 0.999))

In [22]:
def GetGradientNorms(model):
    totalNorm = 0
    for p in model.parameters():
        paramNorm = p.grad.data.norm(2)
        totalNorm += paramNorm.item() ** 2
    return totalNorm ** (1 / 2)

generatorLosses = []
discriminatorLosses = []
generatorGradientNorms = []
discriminatorGradientNorms = []

In [23]:
saveDir = os.path.join(os.getcwd(), "Images")

if not os.path.exists(saveDir):
    os.makedirs(saveDir)

saveDir


'd:\\Finally, A study Folder\\Thapar Summer School on Machine Learning and Deep Learning\\GAN\\DC-GAN\\Images'

In [24]:
fixedNoise = torch.randn(batchSize, 100, 1, 1, device = device)

for epochs in range(numberofEpochs):
    for i, data in enumerate(dataloader, 0):
        realImages, _ = data
        realImages = realImages.to(device)

        optimizerDiscriminator.zero_grad()
        
        output = discriminator(realImages).view(-1)
        realLabels = torch.ones(output.size()).to(device)
        realLoss = criterion(output, realLabels)
        realLoss.backward()

        noise = torch.randn(128, 100, 1, 1, device = device)
        fakeImages = generator(noise)
        output = discriminator(fakeImages.detach()).view(-1)
        fakeLabels = torch.zeros(output.size()).to(device)
        fakeLoss = criterion(output, fakeLabels)
        fakeLoss.backward()
        discriminatorGradientNorms.append(GetGradientNorms(discriminator))
        discriminatorGradientNorm = GetGradientNorms(discriminator)
        optimizerDiscriminator.step()

        optimizerGenerator.zero_grad()
        output = discriminator(fakeImages).view(-1)
        realLabels = torch.ones(output.size()).to(device)
        generatorLoss = criterion(output, realLabels)
        generatorLoss.backward()
        generatorGradientNorms.append(GetGradientNorms(generator))
        generatorGradientNorm = GetGradientNorms(generator)
        optimizerGenerator.step()

        generatorLosses.append(generatorLoss.item())
        discriminatorLosses.append(realLoss.item() + fakeLoss.item())
        generatorGradientNorms.append(generatorGradientNorm)
        discriminatorGradientNorms.append(discriminatorGradientNorm)

    with torch.no_grad():
        fake = generator(fixedNoise).detach().cpu()
    vutils.save_image(fake, f"{saveDir}/fake_{epochs+1}.png", normalize = True)
    print(f"Epochs: {epochs}, Generator Loss: {generatorLoss}, Discriminator Loss: {realLoss + fakeLoss}")


    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(generatorLosses, label='Generator')
    plt.plot(discriminatorLosses, label='Discriminator')
    plt.legend()
    plt.title(f"Epochs: {epochs}, G Loss: {generatorLosses[-1]:.4f}, D Loss: {discriminatorLosses[-1]:.4f}")

    plt.subplot(1, 2, 2)
    plt.plot(generatorGradientNorms, label='Generator')
    plt.plot(discriminatorGradientNorms, label='Discriminator')
    plt.legend()
    plt.title("Gradient Norms")
    plt.savefig(f"{saveDir}/epoch_{epochs}.png")
    plt.close()

RuntimeError: output with shape [1, 28, 28] doesn't match the broadcast shape [3, 28, 28]

In [25]:
Images = [imageio.imread(image) for image in sorted(os.listdir(saveDir))]
imageio.mimsave("generatedImages.gif", [np.array(img) for img in Images], duration=0.5)

ValueError: need at least one array to stack